In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType,ArrayType,LongType
import logging
import sys
import traceback
import ast
import json
import os
import sys
import traceback
import logging
from pyspark.sql.functions import col, expr,when,to_date ,udf, concat_ws,posexplode, from_json
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType
from pyspark.sql import functions as F

In [2]:

spark = SparkSession.builder \
    .appName("HybridRecommenderMaxPerformance") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
    .config("spark.executor.instances", "1") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
    .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("delta.enable-non-concurrent-writes", "true") \
    .config("spark.sql.warehouse.dir", "s3a://lakehouse/") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-76652a8a-dc82-4a06-8fe4-fe3de8d5a071;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

25/06/16 13:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np


ratings = spark.read.format("delta").load("s3a://lakehouse/silver/ratings").cache()
ratings = ratings.select("userId", "movieId", "rating")


25/06/14 18:18:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: float (nullable = true)



In [6]:
from pyspark.ml.recommendation import ALSModel

CF = ALSModel.load("s3a://lakehouse/data/CF_als_best_model")
# Đọc dữ liệu movie_names và ratings_data từ Delta
movies  = spark.read.format("delta").load("s3a://lakehouse/gold/dim_movie")
movies  = movies .select("id", "title")
movies  = movies .withColumnRenamed("id", "movieId")

In [3]:
t  = spark.read.format("delta").load("s3a://lakehouse/data/bertTest")
t.count()

25/06/16 13:57:08 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/06/16 13:57:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
t.count()

45433

In [6]:
a  = spark.read.format("delta").load("s3a://lakehouse/data/bertTestHiHi")
a.count()

29718

25/06/16 14:05:00 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/06/16 14:05:00 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [17]:
from pyspark.sql.functions import col, lit

# select a single user from the test set
user_id = 123
single_user_ratings = ratings.filter(ratings['userId'] == user_id).select(['movieId', 'userId', 'rating'])

# display the movies the user has liked
print("Movies liked by user with ID", user_id)
single_user_ratings.join(movies, 'movieId').select('movieId', 'title', 'rating').show(truncate=False)

# generate recommendations for the user
all_movies = ratings.select('movieId').distinct()
user_movies = single_user_ratings.select('movieId').distinct()
movies_to_recommend = all_movies.subtract(user_movies)

# predict ratings for movies the user has not rated yet
recommendations = CF.transform(movies_to_recommend.withColumn('userId', lit(user_id)))

# filter out the movies that the user has already rated or seen (this filters out the movies that the user has not liked as well)
recommendations = recommendations.filter(col('prediction') > 0)

# display the recommendations with movie names
print("Recommended movies for user with ID", user_id)
recommended_movies = recommendations.join(movies, 'movieId').select('movieId', 'title', 'prediction')

# Sort recommended movies by prediction in descending order
ordered_recommendations = recommended_movies.orderBy(col('prediction').desc())

# Display the ordered recommendations
ordered_recommendations.show(truncate=False)

Movies liked by user with ID 123
+-------+-------------------------------------------------+------+
|movieId|title                                            |rating|
+-------+-------------------------------------------------+------+
|11     |Star Wars                                        |5.0   |
|278    |The Shawshank Redemption                         |3.0   |
|424    |Schindler's List                                 |4.5   |
|562    |Die Hard                                         |4.0   |
|1891   |The Empire Strikes Back                          |5.0   |
|2493   |The Princess Bride                               |5.0   |
|85     |Raiders of the Lost Ark                          |5.0   |
|1892   |Return of the Jedi                               |4.0   |
|137    |Groundhog Day                                    |2.5   |
|105    |Back to the Future                               |4.5   |
|89     |Indiana Jones and the Last Crusade               |4.0   |
|857    |Saving Private Ryan 

[Stage 117:=================================================>       (7 + 1) / 8]

+-------+------------------------------+----------+
|movieId|title                         |prediction|
+-------+------------------------------+----------+
|267506 |UFO - Distruggete base Luna!  |6.095041  |
|84601  |The Wearing of the Grin       |5.9462733 |
|373453 |Brahman Naman                 |5.5657406 |
|189820 |Fifth Avenue, New York        |5.411268  |
|37193  |Hue and Cry                   |5.2873645 |
|52488  |A More Perfect Union          |5.236807  |
|363846 |Motherland                    |5.1648088 |
|143453 |Thrust in Me                  |5.1428375 |
|250275 |Getting That Girl             |5.0788    |
|36952  |Dreams with Sharp Teeth       |5.016851  |
|439314 |The Garden of Afflictions 2017|4.9734693 |
|55739  |Single Santa Seeks Mrs. Claus |4.9212956 |
|156141 |Stalingrad                    |4.918215  |
|48955  |Everybody Go Home             |4.9170036 |
|53419  |Behind the Screen             |4.866202  |
|80809  |Perry Mason Returns           |4.818535  |
|10844  |Out

In [18]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import FloatType, IntegerType
import numpy as np
from pyspark.ml.recommendation import ALSModel
import time
start_time = time.time()
# # Đọc và cache dữ liệu ratings
# ratings = spark.read.format("delta").load("s3a://lakehouse/silver/ratings") \
#     .select("userId", "movieId", "rating") \
#     .dropna(subset=['userId', 'movieId']) \
#     .withColumn('userId', F.col('userId').cast(IntegerType())) \
#     .withColumn('rating', F.col('rating').cast(FloatType())) \
#     .cache()


# ratings = spark.read.format("delta").load("s3a://lakehouse/silver/ratings").cache()
# ratings = ratings.select("userId", "movieId", "rating")

# CF = ALSModel.load("s3a://lakehouse/data/CF_als_best_model")

# Đọc dữ liệu phim và kiểm tra tên phim
movie_name = input("Enter a movie name: ")
df_merge = spark.read.format("delta").load("s3a://lakehouse/gold/MergeData").cache()

movie_df = df_merge.filter(F.col("title") == movie_name).select("id").limit(1)
if movie_df.count() == 0:
    print("Sorry! The movie you searched is not in our database.")
else:
    # Lấy ID phim đầu vào
    m_id = movie_df.collect()[0]['id']

    # Đọc dữ liệu vector phim và sử dụng Broadcast cho vector phim đầu vào
    df_list = spark.read.format("delta").load("s3a://lakehouse/data/bertTestHiHi").cache()
    input_vec = df_list.filter(F.col("id") == m_id).select("vecs").collect()[0][0]
    broadcast_input_vec = spark.sparkContext.broadcast(np.array(input_vec))

    # Định nghĩa UDF tính cosine similarity
    def cosine_similarity(v1, v2):
        v1 = np.array(v1)
        v2 = np.array(v2)
        numerator = float(np.dot(v1, v2))
        denominator = float(np.linalg.norm(v1) * np.linalg.norm(v2))
        return numerator / denominator if denominator != 0 else 0.0

    cosine_udf = F.udf(lambda vec: float(cosine_similarity(broadcast_input_vec.value, vec)), FloatType())

    # Tính toán cosine similarity cho tất cả phim
    recommendations_spark_df = df_list \
        .withColumn("score", cosine_udf(F.col("vecs"))) \
        .filter(F.col("id") != m_id) \
        .select(F.col("id").alias("movies_id"), "score") \
        .orderBy(F.col("score").desc()) \
        .limit(10)

    recommendations_spark_df = recommendations_spark_df.join(
        df_merge,
        F.col("movies_id") == F.col("id"),
        'inner'
    )

    # Chọn các cột mong muốn và hiển thị kết quả
    recommendations_spark_df.select(
        recommendations_spark_df["id"],
        recommendations_spark_df['title'],
        recommendations_spark_df["score"]
    ).orderBy(F.col("score").desc()).show(truncate=False)

    # Hiển thị kết quả
    # recommendations_spark_df.show()
print('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

Enter a movie name:  Toy Story


[Stage 142:>                                                        (0 + 8) / 8]

+------+--------------------------------------+----------+
|id    |title                                 |score     |
+------+--------------------------------------+----------+
|863   |Toy Story 2                           |0.9661569 |
|10193 |Toy Story 3                           |0.9649793 |
|33914 |Frankenweenie                         |0.9564637 |
|62214 |Frankenweenie                         |0.955091  |
|888   |The Flintstones                       |0.9543923 |
|13378 |Mickey's Twice Upon a Christmas       |0.9543722 |
|13397 |The Year Without a Santa Claus        |0.95293456|
|10996 |Stuart Little 2                       |0.9511653 |
|258509|Alvin and the Chipmunks: The Road Chip|0.95008785|
|11114 |Pete's Dragon                         |0.94886434|
+------+--------------------------------------+----------+

Total Runtime: 9.24 seconds


In [23]:
from pyspark.sql.functions import col, lit

# select a single user from the test set
user_id = 12
single_user_ratings = test.filter(test['userId'] == user_id).select(['movieId', 'userId', 'rating'])

# display the movies the user has liked
print("Movies liked by user with ID", user_id)
single_user_ratings.join(movies, 'movieId').select('movieId', 'title', 'rating').show()

# generate recommendations for the user
all_movies = df.select('movieId').distinct()
user_movies = single_user_ratings.select('movieId').distinct()
movies_to_recommend = all_movies.subtract(user_movies)

# predict ratings for movies the user has not rated yet
recommendations = final_model.transform(movies_to_recommend.withColumn('userId', lit(user_id)))

# filter out the movies that the user has already rated or seen (this filters out the movies that the user has not liked as well)
recommendations = recommendations.filter(col('prediction') > 0)

# display the recommendations with movie names
print("Recommended movies for user with ID", user_id)
recommended_movies = recommendations.join(movies, 'movieId').select('movieId', 'title', 'prediction')

# Sort recommended movies by prediction in descending order
ordered_recommendations = recommended_movies.orderBy(col('prediction').desc())

# Display the ordered recommendations
ordered_recommendations.show()

Movies liked by user with ID 12


+-------+--------------------+------+
|movieId|               title|rating|
+-------+--------------------+------+
|    157|Star Trek III: Th...|   1.0|
|    639|When Harry Met Sa...|   3.5|
|   2640|            Heathers|   3.0|
|   8849|               Alfie|   4.0|
|   9340|         The Goonies|   3.0|
|   9715|         Hope Floats|   4.5|
|  11224|          Cinderella|   3.0|
+-------+--------------------+------+

Recommended movies for user with ID 12


+-------+--------------------+----------+
|movieId|               title|prediction|
+-------+--------------------+----------+
| 397403|   Brexit: The Movie|  6.505104|
|   1731|               Helen|  6.163009|
| 393841|      Pelli Choopulu|  5.947569|
| 412758|          Eat Locals| 5.9273725|
| 355177|Tig Notaro: Boyis...| 5.9270086|
| 261810|            Silenced| 5.8784776|
| 409502|     I'm Not Ashamed|  5.839202|
| 228198|Eu Não Faço a Men...|  5.811181|
| 407036|        On the Brain| 5.6171036|
| 461088|50 Kilos of Sour ...| 5.5664444|
|  65488|     Raintree County|  5.538809|
|  56759| Notre Dame de Paris|   5.46525|
|  11496|They Call Me Rene...| 5.4383755|
| 398854|A Midsummer Night...|  5.412646|
| 176088|   The Story of Luke|  5.370575|
|  63039|          Noah's Ark| 5.3683953|
|  42427|           Fair Play|  5.366928|
|  17022|Michael Jackson: ...| 5.3630924|
|  15199|Rent: Filmed Live...|  5.343305|
| 432025|The Art of Loving...|  5.272885|
+-------+--------------------+----

In [ ]:
import numpy as np
df_list = spark.read.format("delta").load("s3a://lakehouse/data/bertTestHi").cache()
df_merge = spark.read.format("delta").load("s3a://lakehouse/gold/MergeData").cache()
# Đọc dữ liệu movie_names và ratings_data từ Delta
movies  = spark.read.format("delta").load("s3a://lakehouse/gold/dim_movie")
movies  = movies .select("id", "title")
movies  = movies .withColumnRenamed("id", "movieId")
from pyspark.ml.recommendation import ALSModel

temp = ALSModel.load("s3a://lakehouse/data/CF_als_best_model")

In [ ]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
import numpy as np

# 1. Chọn 1 user
user_id = 123
single_user_ratings = ratings.filter(ratings['userId'] == user_id).select(['movieId', 'userId', 'rating'])

# 2. In các phim user đã rating
print("Phim user đã xem:")
single_user_ratings.join(df_merge, single_user_ratings['movieId'] == df_merge['id']).select(
    "title", "genres", "director_names", "cast_names", "popularity", "vote_average", "vote_count", "rating").show()

# 3. Lấy danh sách phim chưa xem
all_movies = df.select('movieId').distinct()
user_movies = single_user_ratings.select('movieId').distinct()
movies_to_recommend = all_movies.subtract(user_movies)

# 4. Dự đoán bằng CF
recommendations = temp.transform(movies_to_recommend.withColumn('userId', lit(user_id)))
recommendations = recommendations.filter(col('prediction') > 3.7)

# 5. Join info phim
recommended_movies = recommendations.join(df_merge, recommendations['movieId'] == df_merge['id']).select(
    "title", "genres",  "popularity", "vote_average", 
    "vote_count", "movieId", "prediction"
)

# 6. Lấy vector các phim user đã rating
rated_movies = single_user_ratings.join(df_list, single_user_ratings.movieId == df_list.id)

# 7. Tính vector trung bình
user_vec = rated_movies.select('vecs').rdd.map(lambda row: np.array(row['vecs'])).mean()
broadcast_user_vec = spark.sparkContext.broadcast(user_vec)

# 8. UDF tính cosine similarity
def cosine_similarity(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    numerator = float(np.dot(v1, v2))
    denominator = float(np.linalg.norm(v1) * np.linalg.norm(v2))
    return numerator / denominator if denominator != 0 else 0.0

cosine_udf = F.udf(lambda vec: float(cosine_similarity(broadcast_user_vec.value, vec)), FloatType())

# 9. Join vector vào các phim gợi ý và tính similarity
cb_filtered = recommended_movies.join(df_list, recommended_movies.movieId == df_list.id)
cb_filtered = cb_filtered.withColumn('similarity', cosine_udf(F.col('vecs')))

# 10. Sắp xếp theo similarity
cb_filtered = cb_filtered.orderBy(F.col('similarity').desc())

# 11. Hiển thị kết quả sau lọc bằng CB
print("Phim được lọc lại bằng CB (cosine):")
cb_filtered.select(
    "title", "genres", "popularity", "vote_average", 
    "vote_count", "similarity","prediction"
).show()
